<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250803_3%EC%B0%A8_%ED%85%8C%EC%8A%A4%ED%8A%B8_%EA%B8%B0%EC%97%85%EA%B2%80%EC%83%89_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import requests, zipfile
from io import BytesIO
import xml.etree.ElementTree as ET
import json
import time
import difflib
import openai
from openai import OpenAI

# 🔑 API 키
DART_API_KEY = "your_key"
OPENAI_API_KEY = "your_key"
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ 설정
DEFAULT_YEAR = "2023"
DEFAULT_REPRT_CODE = "11011"


In [85]:

# ✅ 기업 리스트 불러오기
def load_corp_list(api_key):
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    res = requests.get(url)
    z = zipfile.ZipFile(BytesIO(res.content))
    xml_content = z.read(z.namelist()[0])
    root = ET.fromstring(xml_content)
    return [{"corp_code": c.findtext("corp_code"), "corp_name": c.findtext("corp_name")} for c in root.findall("list")]

# ✅ 기업 개요
def collect_profile(corp_code):
    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": DART_API_KEY, "corp_code": corp_code}
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return {}
    return {
        "회사명": res.get("corp_name"),
        "대표자": res.get("ceo_nm"),
        "사업자등록번호": res.get("bizr_no"),
        "설립일": res.get("est_dt"),
        "업종코드": res.get("industry_code"),
        "주소": res.get("adres"),
        "지주회사": res.get("hm_ownr_corp_nm"),
        "상장여부": "상장" if res.get("stock_code") else "비상장"
    }

# ✅ 재무정보
def collect_finance(corp_code):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE,
        "fs_div": "CFS"
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return None
    for item in res.get("list", []):
        if item["account_nm"] == "매출액":
            return item["thstrm_amount"]
    return None

# ✅ 고용정보
def collect_employees(corp_code):
    url = "https://opendart.fss.or.kr/api/emplSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return None
    for item in res.get("list", []):
        if item.get("se") == "합계":
            return item.get("thstrm") or item.get("thstrm_nm")
    return None

# ✅ 계열사
def collect_affiliates(corp_code):
    url = "https://opendart.fss.or.kr/api/dsclSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return []
    return [item.get("corp_name") for item in res.get("list", []) if item.get("corp_name")]

# ✅ GPT 요약
def gpt_summary(profile, sales, emp, affiliates):
    categories = "패션, 물류, 핀테크, 유통, 콘텐츠, 플랫폼, 커머스, IT, 미디어, 제조, 기타"
    keywords = "예시: #2030 #남성패션 #MZ, 주요 사업 아이템이나 고객에 대한 정보 제공"
    investor_types = "#공격형, #안정형, #혼합형"

    prompt = f"""당사는 다음과 같은 기업입니다:

[기업 개요]
- 회사명: {profile.get("회사명")}
- 대표자: {profile.get("대표자")}
- 설립일: {profile.get("설립일")}
- 업종코드: {profile.get("업종코드")}
- 주소: {profile.get("주소")}
- 상장여부: {profile.get("상장여부")}

[재무 및 인력]
- 총매출: {sales or '매출 정보 없음'}
- 총인원 수: {emp or '인력 정보 없음'}

[계열사]
- {', '.join(affiliates) if affiliates else '계열사 정보 없음'}

넌 20년 된 투자 전문가야. 일반인에게 설명할 거야.

1. 이 기업이 속한 주요 산업 분야를 반드시 다음 중 하나로 판단해서 출력해줘 (그 외는 절대 출력하지 마):
{categories}

2. 다음 중 최대 2개의 키워드를 골라 해시태그로 출력해줘 (예:"#2030 #남성패션 #MZ"와 같이 그 기업의 핵심 사업 아이템이나 상황을 잘 드러내는 키워드)],
  "):
{keywords}

3. 반드시 다음 중 하나를 세 번째 키워드로 붙여줘 (항상 마지막에 위치해야 해):
{investor_types}

4. 이 기업을 전문가 관점에서 핵심 요점만 뽑아 한 문장으로 요약해줘.

출력 형식은 다음과 같아야 해:

{{
  "주요 분야": "패션",
  "키워드": [(예:"#2030 #남성패션 #MZ"와 같이 그 기업의 핵심 사업 아이템이나 상황을 잘 드러내는 키워드)],
  "한 문장 요약": "이 기업은 ..."
}}"""
    try:
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        return json.loads(res.choices[0].message.content.strip())
    except Exception as e:
        return {
            "주요 분야": "정보 없음",
            "키워드": ["#정보없음"],
            "한 문장 요약": f"❌ GPT 요약 실패: {e}"
        }

# ✅ 전체 실행
def run_batch_for_keyword(keyword):
    corp_list = load_corp_list(DART_API_KEY)
    keyword_lower = keyword.lower()
    matches = [c for c in corp_list if keyword_lower in c["corp_name"].lower()]
    matches.sort(key=lambda c: (not c["corp_name"].startswith(keyword), c["corp_name"]))
    matches = matches[:20]

    print(f"\n✅ '{keyword}'가 포함된 기업 {len(matches)}개 결과:\n")

    for corp in matches:
        print(f"🔍 {corp['corp_name']} (corp_code: {corp['corp_code']})")
        profile = collect_profile(corp["corp_code"])
        sales = collect_finance(corp["corp_code"])
        emp = collect_employees(corp["corp_code"])
        affiliates = collect_affiliates(corp["corp_code"])
        summary = gpt_summary(profile, sales, emp, affiliates)

        result_dict = {
            "회사명": profile.get("회사명"),
            "대표자": profile.get("대표자"),
            "상장여부": profile.get("상장여부"),
            "설립일": profile.get("설립일"),
            "주소": profile.get("주소"),
            "사업자등록번호": profile.get("사업자등록번호"),
            "총매출(천원)": sales or "정보 없음",
            "총인원 수": emp or "정보 없음",
            "계열사": affiliates if affiliates else [],
            "주요 분야": summary.get("주요 분야"),
            "키워드": summary.get("키워드"),
            "GPT 요약": summary.get("한 문장 요약")
        }

        print(json.dumps(result_dict, indent=2, ensure_ascii=False))
        print("="*100)
        time.sleep(1.2)

# ✅ 실행
if __name__ == "__main__":
    keyword = input("🔍 검색할 기업 키워드를 입력하세요: ")
    run_batch_for_keyword(keyword)


🔍 검색할 기업 키워드를 입력하세요: 로지텍

✅ '로지텍'가 포함된 기업 20개 결과:

🔍 고려로지텍 (corp_code: 00891615)
{
  "회사명": "(주)고려로지텍",
  "대표자": "전용하",
  "상장여부": "비상장",
  "설립일": "20080421",
  "주소": "경상북도 경주시 외동읍 문산리 294번지",
  "사업자등록번호": "5058151769",
  "총매출(천원)": "정보 없음",
  "총인원 수": "정보 없음",
  "계열사": [],
  "주요 분야": "IT",
  "키워드": [
    "#정보기술 #데이터분석",
    "#기술혁신 #빅데이터"
  ],
  "GPT 요약": "이 기업은 IT 기술을 활용한 데이터 분석과 기술 혁신을 주력으로 하는 기업이다. #혼합형"
}
🔍 골드라인로지텍 (corp_code: 00677015)
{
  "회사명": "(주)골드라인로지텍",
  "대표자": "이홍기",
  "상장여부": "비상장",
  "설립일": "19930315",
  "주소": "경기도 수원시 영통구 신원로294번길 38 (원천동)",
  "사업자등록번호": "2208116691",
  "총매출(천원)": "정보 없음",
  "총인원 수": "정보 없음",
  "계열사": [],
  "주요 분야": "IT",
  "키워드": [
    "#기술혁신 #데이터분석",
    "#안정형"
  ],
  "GPT 요약": "이 기업은 IT 기술을 통해 안정적으로 성장하고 있으며 데이터 분석을 통해 혁신을 이끌고 있다."
}
🔍 국양로지텍 (corp_code: 01135686)
{
  "회사명": "국양로지텍(주)",
  "대표자": "금창원",
  "상장여부": "비상장",
  "설립일": "19910101",
  "주소": "부산광역시 중구 충장대로 11 (중앙동4가, 무역회관빌딩)",
  "사업자등록번호": "1048144880",
  "총매출(천원)": "정보 없음",
  "총인원 수": "정보 없음",


KeyboardInterrupt: 

In [89]:
def convert_result_to_react_json(result_dict, image_url=None):
    return {
        "name": result_dict["회사명"],
        "src": image_url or "",  # 썸네일 URL 없으면 비워둬도 됨
        "itemList": [
            {
                "type": "text",
                "data": [result_dict["GPT 요약"]]
            },
            {
                "type": "text",
                "data": [result_dict["상장여부"]]
            },
            {
                "type": "text",
                "data": [f"{int(result_dict['총매출(천원)'])/1e8:.0f}억" if result_dict['총매출(천원)'] not in ["정보 없음", None] else "정보 없음"]
            },
            {
                "type": "text",
                "data": [f"{result_dict['총인원 수']}명" if result_dict['총인원 수'] not in ["정보 없음", None] else "정보 없음"]
            },
            {
                "type": "tag",
                "data": [result_dict["주요 분야"]] + result_dict.get("키워드", [])[:-1]  # 투자자 성향(#공격형)은 마지막에 있어서 제외
            },
            {
                "type": "tag",
                "data": result_dict.get("키워드", [])
            }
        ]
    }


NameError: name 'result_dict' is not defined

# 이하 연습용 코드들

In [81]:
# # ✅ 기업 리스트 불러오기
# def load_corp_list(api_key):
#     url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
#     res = requests.get(url)
#     z = zipfile.ZipFile(BytesIO(res.content))
#     xml_content = z.read(z.namelist()[0])
#     root = ET.fromstring(xml_content)
#     data = []
#     for child in root.findall("list"):
#         data.append({
#             "corp_code": child.findtext("corp_code"),
#             "corp_name": child.findtext("corp_name")
#         })
#     return data

# # ✅ 기업 개요
# def collect_profile(corp_code):
#     url = "https://opendart.fss.or.kr/api/company.json"
#     params = {
#         "crtfc_key": DART_API_KEY,
#         "corp_code": corp_code
#     }
#     res = requests.get(url, params=params).json()
#     if res.get("status") != "000":
#         return {}
#     return {
#         "회사명": res.get("corp_name"),
#         "대표자": res.get("ceo_nm"),
#         "사업자등록번호": res.get("bizr_no"),
#         "설립일": res.get("est_dt"),
#         "업종코드": res.get("industry_code"),
#         "주소": res.get("adres"),
#         "지주회사": res.get("hm_ownr_corp_nm"),
#         "상장여부": "상장" if res.get("stock_code") else "비상장"
#     }

# # ✅ 재무정보
# def collect_finance(corp_code):
#     url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
#     params = {
#         "crtfc_key": DART_API_KEY,
#         "corp_code": corp_code,
#         "bsns_year": DEFAULT_YEAR,
#         "reprt_code": DEFAULT_REPRT_CODE,
#         "fs_div": "CFS"
#     }
#     res = requests.get(url, params=params).json()
#     if res.get("status") != "000":
#         return None
#     for item in res.get("list", []):
#         if item["account_nm"] == "매출액":
#             return item["thstrm_amount"]
#     return None

# # ✅ 고용정보
# def collect_employees(corp_code):
#     url = "https://opendart.fss.or.kr/api/emplSttus.json"
#     params = {
#         "crtfc_key": DART_API_KEY,
#         "corp_code": corp_code,
#         "bsns_year": DEFAULT_YEAR,
#         "reprt_code": DEFAULT_REPRT_CODE
#     }
#     res = requests.get(url, params=params).json()
#     if res.get("status") != "000":
#         return None
#     for item in res.get("list", []):
#         if item.get("se") == "합계":
#             return item.get("thstrm") or item.get("thstrm_nm")
#     return None

# # ✅ 계열사
# def collect_affiliates(corp_code):
#     url = "https://opendart.fss.or.kr/api/dsclSttus.json"
#     params = {
#         "crtfc_key": DART_API_KEY,
#         "corp_code": corp_code,
#         "bsns_year": DEFAULT_YEAR,
#         "reprt_code": DEFAULT_REPRT_CODE
#     }
#     res = requests.get(url, params=params).json()
#     if res.get("status") != "000":
#         return []
#     return [item.get("corp_name") for item in res.get("list", []) if item.get("corp_name")]

# # ✅ GPT 요약
# def gpt_summary(profile, sales, emp, affiliates):
#     prompt = f"""당사는 다음과 같은 기업입니다:

# [기업 개요]
# - 회사명: {profile.get("회사명")}
# - 대표자: {profile.get("대표자")}
# - 설립일: {profile.get("설립일")}
# - 업종코드: {profile.get("업종코드")}
# - 주소: {profile.get("주소")}
# - 상장여부: {profile.get("상장여부")}

# [재무 및 인력]
# - 총매출: {sales or '매출 정보 없음'}
# - 총인원 수: {emp or '인력 정보 없음'}

# [계열사]
# - {', '.join(affiliates) if affiliates else '계열사 정보 없음'}

# 이 정보를 바탕으로, 이 기업이 어떤 산업에 속해 있고 어떤 특징이 있는지 한 문단으로 요약해줘."""
#     try:
#         res = client.chat.completions.create(
#             model="gpt-3.5-turbo",
#             messages=[{"role": "user", "content": prompt}],
#             temperature=0.5
#         )
#         return res.choices[0].message.content.strip()
#     except Exception as e:
#         return f"❌ GPT 요약 실패: {e}"

# # ✅ 전체 실행
# def run_batch_for_keyword(keyword):
#     corp_list = load_corp_list(DART_API_KEY)
#     keyword_lower = keyword.lower()

#     # 키워드 포함 기업만 필터링
#     matches = [c for c in corp_list if keyword_lower in c["corp_name"].lower()]

#     # 키워드로 시작하는 기업 우선 정렬
#     matches.sort(key=lambda c: (not c["corp_name"].startswith(keyword), c["corp_name"]))

#     # 상위 20개만 유지
#     matches = matches[:20]

#     print(f"\n✅ '{keyword}'가 포함된 기업 {len(matches)}개 결과:\n")

#     for corp in matches:
#         print(f"🔍 {corp['corp_name']} (corp_code: {corp['corp_code']})")
#         profile = collect_profile(corp["corp_code"])
#         sales = collect_finance(corp["corp_code"])
#         emp = collect_employees(corp["corp_code"])
#         affiliates = collect_affiliates(corp["corp_code"])
#         summary = gpt_summary(profile, sales, emp, affiliates)

#         result_dict = {
#             "회사명": profile.get("회사명"),
#             "대표자": profile.get("대표자"),
#             "상장여부": profile.get("상장여부"),
#             "설립일": profile.get("설립일"),
#             "주소": profile.get("주소"),
#             "사업자등록번호": profile.get("사업자등록번호"),
#             "총매출(천원)": sales or "정보 없음",
#             "총인원 수": emp or "정보 없음",
#             "계열사": affiliates if affiliates else [],
#             "GPT요약": summary
#         }

#         print(json.dumps(result_dict, indent=2, ensure_ascii=False))
#         print("="*100)
#         time.sleep(1.2)

# # ✅ 실행
# keyword = input("🔍 검색할 기업 키워드를 입력하세요: ")
# run_batch_for_keyword(keyword)

## 250803 수정


In [69]:
# ✅ 기업 리스트 불러오기
def load_corp_list(api_key):
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    res = requests.get(url)
    z = zipfile.ZipFile(BytesIO(res.content))
    xml_content = z.read(z.namelist()[0])
    root = ET.fromstring(xml_content)
    return [{"corp_code": c.findtext("corp_code"), "corp_name": c.findtext("corp_name")} for c in root.findall("list")]

# ✅ 기업 개요
def collect_profile(corp_code):
    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": DART_API_KEY, "corp_code": corp_code}
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return {}
    return {
        "회사명": res.get("corp_name"),
        "대표자": res.get("ceo_nm"),
        "사업자등록번호": res.get("bizr_no"),
        "설립일": res.get("est_dt"),
        "업종코드": res.get("industry_code"),
        "주소": res.get("adres"),
        "지주회사": res.get("hm_ownr_corp_nm"),
        "상장여부": "상장" if res.get("stock_code") else "비상장"
    }

# ✅ 재무정보
def collect_finance(corp_code):
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE,
        "fs_div": "CFS"
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return None
    for item in res.get("list", []):
        if item["account_nm"] == "매출액":
            return item["thstrm_amount"]
    return None

# ✅ 고용정보
def collect_employees(corp_code):
    url = "https://opendart.fss.or.kr/api/emplSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return None
    for item in res.get("list", []):
        if item.get("se") == "합계":
            return item.get("thstrm") or item.get("thstrm_nm")
    return None

# ✅ 계열사
def collect_affiliates(corp_code):
    url = "https://opendart.fss.or.kr/api/dsclSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return []
    return [item.get("corp_name") for item in res.get("list", []) if item.get("corp_name")]

# ✅ GPT 요약
def gpt_summary(profile, sales, emp, affiliates):
    categories = "패션, 물류, 핀테크, 유통, 콘텐츠, 플랫폼, 커머스, IT, 미디어, 제조, 기타"
    investors = "혼합형, 공격형, 안정형"
    prompt = f"""당사는 다음과 같은 기업입니다:

[기업 개요]
- 회사명: {profile.get("회사명")}
- 대표자: {profile.get("대표자")}
- 설립일: {profile.get("설립일")}
- 업종코드: {profile.get("업종코드")}
- 주소: {profile.get("주소")}
- 상장여부: {profile.get("상장여부")}

[재무 및 인력]
- 총매출: {sales or '매출 정보 없음'}
- 총인원 수: {emp or '인력 정보 없음'}

[계열사]
- {', '.join(affiliates) if affiliates else '계열사 정보 없음'}

넌 20년된 투자전문가야. 일반인들에게 투자할 가치가 있는 기업을 설명할 거야.
1. 다음 중 하나 이상을 골라 이 기업을 설명하는 최대 3개의 키워드를 해시태그 형식으로 출력해줘 (반드시 이 중에서만 골라서 #으로 시작하게 작성해줘):
{categories}, {investors}, 상장기업, 비상장기업, 스타트업, 대기업, 중소기업

2. 이 기업이 어떤 특징을 가지는지 전문가 입장에서 핵심적인 요소를 추려 쉽게 한 문장으로 요약해줘.

3. 출력은 다음과 같은 JSON 형식으로 해줘:
{{
  "키워드": ["#패션", "#"],
  "한 문장 요약": "이 기업은 ..."
}}"""
    try:
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        return json.loads(res.choices[0].message.content.strip())
    except Exception as e:
        return {
            "키워드": ["#정보없음"],
            "한 문장 요약": f"❌ GPT 요약 실패: {e}"
        }

# ✅ 전체 실행
def run_batch_for_keyword(keyword):
    corp_list = load_corp_list(DART_API_KEY)
    keyword_lower = keyword.lower()
    matches = [c for c in corp_list if keyword_lower in c["corp_name"].lower()]
    matches.sort(key=lambda c: (not c["corp_name"].startswith(keyword), c["corp_name"]))
    matches = matches[:20]

    print(f"\n✅ '{keyword}'가 포함된 기업 {len(matches)}개 결과:\n")

    for corp in matches:
        print(f"🔍 {corp['corp_name']} (corp_code: {corp['corp_code']})")
        profile = collect_profile(corp["corp_code"])
        sales = collect_finance(corp["corp_code"])
        emp = collect_employees(corp["corp_code"])
        affiliates = collect_affiliates(corp["corp_code"])
        summary = gpt_summary(profile, sales, emp, affiliates)

        result_dict = {
            "회사명": profile.get("회사명"),
            "대표자": profile.get("대표자"),
            "상장여부": profile.get("상장여부"),
            "설립일": profile.get("설립일"),
            "주소": profile.get("주소"),
            "사업자등록번호": profile.get("사업자등록번호"),
            "총매출(천원)": sales or "정보 없음",
            "총인원 수": emp or "정보 없음",
            "계열사": affiliates if affiliates else [],
            "키워드": summary.get("키워드"),
            "GPT 요약": summary.get("한 문장 요약")
        }

        print(json.dumps(result_dict, indent=2, ensure_ascii=False))
        print("="*100)
        time.sleep(1.2)

# ✅ 실행
if __name__ == "__main__":
    keyword = input("🔍 검색할 기업 키워드를 입력하세요: ")
    run_batch_for_keyword(keyword)


🔍 검색할 기업 키워드를 입력하세요: 무신사

✅ '무신사'가 포함된 기업 5개 결과:

🔍 무신사 (corp_code: 01137727)
{
  "회사명": "주식회사 무신사",
  "대표자": "한문일",
  "상장여부": "비상장",
  "설립일": "20120625",
  "주소": "서울특별시 성동구 아차산로13길 11 (성수동2가) 무신사캠퍼스 N1 1층",
  "사업자등록번호": "2118879575",
  "총매출(천원)": "정보 없음",
  "총인원 수": "정보 없음",
  "계열사": [],
  "키워드": [
    "#패션",
    "#커머스",
    "#스타트업"
  ],
  "GPT 요약": "이 스타트업 기업은 온라인 패션 커머스 플랫폼을 운영하며 새로운 트렌드를 선도하는 역할을 하고 있습니다."
}
🔍 무신사로지스틱스 (corp_code: 01544149)
{
  "회사명": "주식회사 무신사로지스틱스 ",
  "대표자": "이현석",
  "상장여부": "비상장",
  "설립일": "20171113",
  "주소": "경기도 여주시 대신면 여양로 1303 (초현리)",
  "사업자등록번호": "5648800744",
  "총매출(천원)": "정보 없음",
  "총인원 수": "정보 없음",
  "계열사": [],
  "키워드": [
    "#물류",
    "#비상장기업"
  ],
  "GPT 요약": "이 기업은 물류 업종에서 비상장기업으로 운영되고 있습니다."
}
🔍 무신사트레이딩 (corp_code: 01545175)
{
  "회사명": "주식회사 무신사트레이딩",
  "대표자": "이지훈",
  "상장여부": "비상장",
  "설립일": "20100104",
  "주소": "서울특별시 강남구 선릉로93길 35 3층(역삼동, 나라키움B빌딩)",
  "사업자등록번호": "2118837444",
  "총매출(천원)": "정보 없음",
  "총인원 수": "정보 없음",
  "계열사": [],
  "키워드": [
    "